## Imports

In [1]:
from IPython.display import Math
from lib3d_mec_ginac import *
from math import pi, e

Gravity vector

In [3]:
g = get_param('g')

In [4]:
gravity_vector = Vector3D(0, 0, g, get_base('xyz')) # change to -g (no ops implemented)

In [5]:
gravity_vector

[ 0, 0, g ]

In [6]:
 # Integration step
delta_t =             .003
# Assembly init problem solver parameters
geom_eq_init_tol =    1e-10
geom_eq_init_relax =  .1
# Assembly problem solver parameters
geom_eq_tol =         delta_t**2 * 10**-3
geom_eq_relax =       .1
# Equilibrium problem solver parameters
dyn_eq_tol =          1.0e-10
dyn_eq_relax =        .1
# Perturbed dynamic state solver parameters
per_dyn_state_tol =   1e-12
# ...

Generalized coordinates, velocities and accelerations

In [7]:
theta1, dtheta1, ddtheta1 = new_coord('theta1', -pi/6, 0)
theta2, dtheta2, ddtheta2 = new_coord('theta2', -2*pi/6, 0)
theta3, dtheta3, ddtheta3 = new_coord('theta3', -3*pi/6, 0)

In [8]:
get_coordinates()

theta1  -0.523599
theta2  -1.0472
theta3  -1.5708

In [9]:
get_velocities()

dtheta1  0
dtheta2  0
dtheta3  0

In [10]:
get_accelerations()

ddtheta1  0
ddtheta2  0
ddtheta3  0

Geometric parameters

In [11]:
l1, l2 = new_param('l1', 0.4), new_param('l2', 2.0)
l3, l4 = new_param('l3', 1.2), new_param('l4', 1.6)

In [12]:
new_base('Barm1', 'xyz', [0, 1, 0], theta1)
new_base('Barm2', 'xyz', 0, 1, 0, theta2)
new_base('Barm3', 'xyz', rotation_tupla=[0, 1, 0], rotation_angle=theta3);

In [13]:
OA = new_vector('O_A', l1, 0, 0, 'Barm1')
AB = new_vector('A_B', l2, 0, 0, 'Barm2')
BC = new_vector('B_C', [l3, 0, 0], 'Barm3')
OO2 = new_vector('OO2', values=[l4, 0, 0], base='xyz')

In [14]:
new_point('OA', 'O', OA)
new_point('OB', 'OA', AB)
new_point('OC', 'OB', BC)
new_point('O2', 'O', OO2);

In [15]:
m1, m2, m3 = new_param('m1', 1), new_param('m2', 1), new_param('m3', 1)

In [16]:
cg1x, cg1z = new_param('cg1x', 0.2), new_param('cg1z', 0.1)
cg2x, cg2z = new_param('cg2x', 1),   new_param('cg2z', 0.1)
cg3x, cg3z = new_param('cg3x', 0.6), new_param('cg3z', 0.1)

In [17]:
OArm1GArm1 = new_vector('OArm1_Garm1', cg1x, 0, cg1z, 'Barm1')
OArm2GArm2 = new_vector('OArm2_GArm2', cg2x, 0, cg2z, 'Barm2')
OArm3GArm3 = new_vector('OArm3_GArm3', cg3x, 0, cg3z, 'Barm3')

In [18]:
I1yy, I2yy, I3yy = [new_param(name, 1) for name in ('I1yy', 'I2yy', 'I3yy')]

In [19]:
sys.symbols

name      type              value
theta1    coordinate    -0.523599
theta2    coordinate    -1.0472
theta3    coordinate    -1.5708
dtheta1   velocity       0
dtheta2   velocity       0
dtheta3   velocity       0
ddtheta1  acceleration   0
ddtheta2  acceleration   0
ddtheta3  acceleration   0
g         parameter      9.8
l1        parameter      0.4
l2        parameter      2
l3        parameter      1.2
l4        parameter      1.6
m1        parameter      1
m2        parameter      1
m3        parameter      1
cg1x      parameter      0.2
cg1z      parameter      0.1
cg2x      parameter      1
cg2z      parameter      0.1
cg3x      parameter      0.6
cg3z      parameter      0.1
I1yy      parameter      1
I2yy      parameter      1
I3yy      parameter      1

In [20]:
sys.bases

xyz
 +-- Barm1
 +-- Barm2
 +-- Barm3

In [24]:
sys.points

name    x    y    z    base    previous
O
OA      l1   0    0    Barm1   O
OB      l2   0    0    Barm2   OA
OC      l3   0    0    Barm3   OB
O2      l4   0    0    xyz     O

In [25]:
sys.vectors

name         x     y    z     base
O_A          l1    0    0     Barm1
A_B          l2    0    0     Barm2
B_C          l3    0    0     Barm3
OO2          l4    0    0     xyz
OArm1_Garm1  cg1x  0    cg1z  Barm1
OArm2_GArm2  cg2x  0    cg2z  Barm2
OArm3_GArm3  cg3x  0    cg3z  Barm3

In [29]:
OArm1GArm1.skew

╭                    ╮
│    0  -cg1z      0 │
│ cg1z      0  -cg1x │
│    0   cg1x      0 │
╰                    ╯

In [34]:
OArm1GArm1.skew.transpose()

╭                    ╮
│     0   cg1z     0 │
│ -cg1z      0  cg1x │
│     0  -cg1x     0 │
╰                    ╯

In [36]:
OArm1GArm1.module

(cg1z**2+cg1x**2)**(1/2)